# Campaign Finance Disclosure Filings

## Setup

In [2]:
%load_ext sql

In [3]:
from django.conf import settings
connection_string = 'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{NAME}'.format(
    **settings.DATABASES['default']
)
%sql $connection_string

u'Connected: ccdc@calaccess_website'

## Cover Sheets

Every campaign finance disclosure filing has a cover sheet, and the top-level information from these cover sheets ends up in the `CVR_CAMPAIGN_DISCLOSURE_CD`. This is an important table because it links because it links the dollar amount totals from the `SMRY_CD` table to a name fields and a unique identifier of the filer.

The [forms](http://calaccess.californiacivicdata.org/documentation/calaccess-files/cvr-campaign-disclosure-cd/#forms) of the filings that end up in `CVR_CAMPAIGN_DISCLOSURE_CD` are all the ones that include financial disclosures of campaigns as opposed to statements of intention and organization.

### Do the `FORM_TYPE` values vary between amendments to the same any campaign filing?

No, which is good because we can easily sort records linked to the `FILING_ID` and `AMEND_ID` combinations.

In [4]:
%%sql
SELECT cvr."FILING_ID", COUNT(DISTINCT cvr."FORM_TYPE")
FROM "CVR_CAMPAIGN_DISCLOSURE_CD" cvr
JOIN "FILER_FILINGS_CD" ff
ON cvr."FILING_ID" = ff."FILING_ID"
AND cvr."AMEND_ID" = ff."FILING_SEQUENCE"
GROUP BY 1
HAVING COUNT(DISTINCT cvr."FORM_TYPE") > 1;

0 rows affected.


FILING_ID,count


### Do the `FILER_ID` values vary between amendments to the same any campaign filing?

No, which is good because we should then be able to sort out who filed which filings.

In [5]:
%%sql
SELECT "FILING_ID", COUNT(DISTINCT "FILER_ID")
FROM "CVR_CAMPAIGN_DISCLOSURE_CD"
GROUP BY 1
HAVING COUNT(DISTINCT "FILER_ID") > 1
ORDER BY COUNT(DISTINCT "FILER_ID") DESC;

0 rows affected.


FILING_ID,count


## Joining to `FILER_FILINGS_CD`

The `FILER_FILINGS_CD` has additional information about each filing, including the filing period in which the filing files. 

There are also lot of fields that seem to be redundant with fields on `CVR_CAMPAIGN_DISCLOSURE_CD`, specifically:
* `CVR_CAMPAIGN_DISCLOSURE_CD.FORM_TYPE` and `FILER_FILINGS_CD.FORM_ID`
* `CVR_CAMPAIGN_DISCLOSURE_CD.FILER_ID` and `FILER_FILINGS_CD.FILER_ID`
* `CVR_CAMPAIGN_DISCLOSURE_CD.STMT_TYPE` and `FILER_FILINGS_CD.STMNT_TYPE`
* `CVR_CAMPAIGN_DISCLOSURE_CD.RPT_DATE` and `FILER_FILINGS_CD.FILING_DATE`
* `CVR_CAMPAIGN_DISCLOSURE_CD.FROM_START` and `FILER_FILINGS_CD.RPT_START`
* `CVR_CAMPAIGN_DISCLOSURE_CD.THRU_DATE` and `FILER_FILINGS_CD.RPT_END`
* `CVR_CAMPAIGN_DISCLOSURE_CD.RPT_DATE` and `FILER_FILINGS_CD.RPT_DATE`

Might be worth checking if values in each pair of fields ever conflict.

### Does every `CVR_CAMPAIGN_DISCLOSURE_CD` record have a `FILER_FILINGS_CD` record?

Almost. And among records left on the `FILER_FILINGS_CD` table, many of the fields or blank of have values that suggest they are only for testing purposes.

In [6]:
%%sql
SELECT cvr."FORM_TYPE", cvr."FILING_ID", cvr."AMEND_ID", cvr."FILER_ID", cvr."FILER_NAML", cvr."RPT_DATE", *
FROM "CVR_CAMPAIGN_DISCLOSURE_CD" cvr
LEFT JOIN "FILER_FILINGS_CD" ff
ON cvr."FILING_ID" = ff."FILING_ID"
AND cvr."AMEND_ID" = ff."FILING_SEQUENCE"
WHERE ff."FILING_ID" IS NULL or ff."FILING_SEQUENCE" IS NULL;

37 rows affected.


FORM_TYPE,FILING_ID,AMEND_ID,FILER_ID,FILER_NAML,RPT_DATE,id,AMEND_ID_1,AMENDEXP_1,AMENDEXP_2,AMENDEXP_3,ASSOC_CB,ASSOC_INT,BAL_ID,BAL_JURIS,BAL_NAME,BAL_NUM,BRDBASE_YN,BUS_CITY,BUS_INTER,BUS_NAME,BUS_ST,BUS_ZIP4,BUSACT_CB,BUSACTVITY,CAND_CITY,CAND_EMAIL,CAND_FAX,CAND_ID,CAND_NAMF,CAND_NAML,CAND_NAMS,CAND_NAMT,CAND_PHON,CAND_ST,CAND_ZIP4,CMTTE_ID,CMTTE_TYPE,CONTROL_YN,DIST_NO,ELECT_DATE,EMPLBUS_CB,EMPLOYER,ENTITY_CD,FILE_EMAIL,FILER_CITY,FILER_FAX,FILER_ID_1,FILER_NAMF,FILER_NAML_1,FILER_NAMS,FILER_NAMT,FILER_PHON,FILER_ST,FILER_ZIP4,FILING_ID_1,FORM_TYPE_1,FROM_DATE,JURIS_CD,JURIS_DSCR,LATE_RPTNO,MAIL_CITY,MAIL_ST,MAIL_ZIP4,OCCUPATION,OFF_S_H_CD,OFFIC_DSCR,OFFICE_CD,OTHER_CB,OTHER_INT,PRIMFRM_YN,REC_TYPE,REPORT_NUM,REPORTNAME,RPT_ATT_CB,RPT_DATE_1,RPTFROMDT,RPTTHRUDT,SELFEMP_CB,SPONSOR_YN,STMT_TYPE,SUP_OPP_CD,THRU_DATE,TRES_CITY,TRES_EMAIL,TRES_FAX,TRES_NAMF,TRES_NAML,TRES_NAMS,TRES_NAMT,TRES_PHON,TRES_ST,TRES_ZIP4,id_1,FILER_ID_2,FILING_ID_2,PERIOD_ID,FORM_ID,FILING_SEQUENCE,FILING_DATE,STMNT_TYPE,STMNT_STATUS,SESSION_ID,USER_ID,SPECIAL_AUDIT,FINE_AUDIT,RPT_START,RPT_END,RPT_DATE_2,FILING_TYPE
F460,591533,1,992218,Damian Jones for Assembly,2000-07-30,35063484,1,Additional information received after filing.,,,,,,,,,N,,,,,,,,Pasadena,,,,Damian,Jones,,,626/398-4993,CA,91107,,C,0,44,2000-03-07,,,CTL,,Pasadena,,992218,,Damian Jones for Assembly,,,626/449-3346,CA,91107,591533,F460,2000-01-23,ASM,,,Pasadena,CA,911020445,,,,ASM,,,N,CVR,001,,,2000-07-30,None,None,,0,PE,,2000-02-19,Elk Grove,,,Vona,Copp,,,916/686-1815,CA,95624,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
F497,595429,0,990168,X,2000-02-24,35056717,0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,,None,,,,,,,990168,,X,,,,,,595429,F497,None,,,000551,,,,,,,,,,,CVR,000,,,2000-02-24,None,None,,None,,,None,,,,,,,,,,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
F497,598450,0,990168,x,2000-02-28,35058478,0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,,None,,,,,,,990168,,x,,,,,,598450,F497,None,,,002251,,,,,,,,,,,CVR,000,,,2000-02-28,None,None,,None,,,None,,,,,,,,,,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
F497,601406,0,990168,X,2000-02-29,35061219,0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,,None,,,,,,,990168,,X,,,,,,601406,F497,None,,,002371,,,,,,,,,,,CVR,000,,,2000-02-29,None,None,,None,,,None,,,,,,,,,,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
F497,602607,0,990168,X,2000-03-01,35057714,0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,,None,,,,,,,990168,,X,,,,,,602607,F497,None,,,003120,,,,,,,,,,,CVR,000,,,2000-03-01,None,None,,None,,,None,,,,,,,,,,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
F460,602619,1,931704,United Teachers Los Angeles-Political Action Council of Educators(PACE)Issues,2000-02-24,35059163,1,The amount of $1071.95 and $1063.67 were included in Schedule E but not included in Schedule D,,,,,,,,,Y,,,,,,,,,,,,,,,,,,,,G,0,,2000-03-07,,,RCP,,Los Angeles,,931704,,United Teachers Los Angeles-Political Action Council of Educators(PACE)Issues,,,213-487-5560,CA,90010,602619,F460,2000-01-01,,,,,,,,,,,,,N,CVR,001,,,2000-02-24,None,None,,1,PE,,2000-02-19,Los Angeles,,,Patricia,Stanyo,,,213-487-5560,CA,90010,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
F460,670063,1,950027,Carl Washington For CA St Assembly 52nd District,2000-10-26,35065651,1,Schedule F See Attached,,,,,,,,,N,,,,,,,,Paramount,,,,Carl,Washington,,,,CA,90723,,C,0,52,2000-11-07,,,CAO,,Paramount,,950027,-,Carl Washington For CA St Assembly 52nd District,,,,CA,90723,670063,F460,2000-07-01,ASM,State of California,,Paramount,CA,90723-0000,,H,Assemblymember,ASM,,,N,CVR,1,,,2000-10-26,None,None,,0,PE,S,2000-09-30,Los Angeles,,,Pam,Goodwin,,,(310) 223-0759,CA,90037,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
F497,682833,0,972148,SHELLEY FOR ASSEMBLY,2000-10-28,35067303,

### Do any records have conflicting `CVR_CAMPAIGN_DISCLOSURE_CD.FORM_TYPE` and `FILER_FILINGS_CD.FORM_ID` values?

They do, but not a lot. And there is a pretty clear pattern with `CVR_CAMPAIGN_DISCLOSURE_CD.FORM_TYPE` saying `F497` and `FILER_FILINGS_CD.FORM_ID` saying `F496`.

In [7]:
%%sql
SELECT cvr."FORM_TYPE", ff."FORM_ID", cvr."FILING_ID", cvr."AMEND_ID", cvr."RPT_DATE", *
FROM "CVR_CAMPAIGN_DISCLOSURE_CD" cvr
JOIN "FILER_FILINGS_CD" ff
ON cvr."FILING_ID" = ff."FILING_ID"
AND cvr."AMEND_ID" = ff."FILING_SEQUENCE"
WHERE UPPER(cvr."FORM_TYPE") <> UPPER(ff."FORM_ID")
ORDER BY cvr."RPT_DATE" DESC, cvr."FILING_ID" DESC, cvr."AMEND_ID" DESC;

113 rows affected.


FORM_TYPE,FORM_ID,FILING_ID,AMEND_ID,RPT_DATE,id,AMEND_ID_1,AMENDEXP_1,AMENDEXP_2,AMENDEXP_3,ASSOC_CB,ASSOC_INT,BAL_ID,BAL_JURIS,BAL_NAME,BAL_NUM,BRDBASE_YN,BUS_CITY,BUS_INTER,BUS_NAME,BUS_ST,BUS_ZIP4,BUSACT_CB,BUSACTVITY,CAND_CITY,CAND_EMAIL,CAND_FAX,CAND_ID,CAND_NAMF,CAND_NAML,CAND_NAMS,CAND_NAMT,CAND_PHON,CAND_ST,CAND_ZIP4,CMTTE_ID,CMTTE_TYPE,CONTROL_YN,DIST_NO,ELECT_DATE,EMPLBUS_CB,EMPLOYER,ENTITY_CD,FILE_EMAIL,FILER_CITY,FILER_FAX,FILER_ID,FILER_NAMF,FILER_NAML,FILER_NAMS,FILER_NAMT,FILER_PHON,FILER_ST,FILER_ZIP4,FILING_ID_1,FORM_TYPE_1,FROM_DATE,JURIS_CD,JURIS_DSCR,LATE_RPTNO,MAIL_CITY,MAIL_ST,MAIL_ZIP4,OCCUPATION,OFF_S_H_CD,OFFIC_DSCR,OFFICE_CD,OTHER_CB,OTHER_INT,PRIMFRM_YN,REC_TYPE,REPORT_NUM,REPORTNAME,RPT_ATT_CB,RPT_DATE_1,RPTFROMDT,RPTTHRUDT,SELFEMP_CB,SPONSOR_YN,STMT_TYPE,SUP_OPP_CD,THRU_DATE,TRES_CITY,TRES_EMAIL,TRES_FAX,TRES_NAMF,TRES_NAML,TRES_NAMS,TRES_NAMT,TRES_PHON,TRES_ST,TRES_ZIP4,id_1,FILER_ID_1,FILING_ID_2,PERIOD_ID,FORM_ID_1,FILING_SEQUENCE,FILING_DATE,STMNT_TYPE,STMNT_STATUS,SESSION_ID,USER_ID,SPECIAL_AUDIT,FINE_AUDIT,RPT_START,RPT_END,RPT_DATE_2,FILING_TYPE
F497,F496,687938,0,2000-11-07,35070140,0,,,,,,,,,,,,,,,,,,ANTIOCH,,,095057,TOM,TORLAKSON,,,5103727990,CA,945090000,,,None,07,None,,,RCP,,SAN FRANCISCO,,741504,,"STANDING COMMITTEE ON POLITICAL EDUCATION OF THE CALIFORNIA LABOR FEDERATION, AFL-CIO",,,4159863585,CA,941040000,687938,F497,None,,,,,,,,S,,SEN,,,,CVR,0,,,2000-11-07,None,None,,None,,S,None,,,,,,,,,,,183182717,1018355,687938,None,F496,0,2000-11-07,10001,11001,1999,GPEREZ,11003,11003,None,None,2000-11-07,None
F497,F496,687359,0,2000-11-07,35069758,0,,,,,,,,,,,,,,,,,,ETIWANDA,,,089034,JAMES L.,BRULTE,,,9094669096,CA,917390000,,,None,31,None,,,RCP,,LOS ANGELES,,741666,,DEMOCRATIC STATE CENTRAL COMMITTEE OF CALIFORNIA,,,2132561968,CA,900410000,687359,F497,None,,,,,,,,S,,SEN,,,,CVR,0,,,2000-11-07,None,None,,None,,O,None,,,,,,,,,,,182671555,1018392,687359,None,F496,0,2000-11-07,10001,11001,1999,GPEREZ,11003,11003,None,None,2000-11-07,None
F497,F496,687358,0,2000-11-07,35069468,0,,,,,,,,,,,,,,,,,,ANTIOCH,,,095057,TOM,TORLAKSON,,,5103727990,CA,945090000,,,None,07,None,,,RCP,,SAN FRANCISCO,,741504,,"STANDING COMMITTEE ON POLITICAL EDUCATION OF THE CALIFORNIA LABOR FEDERATION, AFL-CIO",,,4159863585,CA,941040000,687358,F497,None,,,,,,,,S,,SEN,,,,CVR,0,,,2000-11-07,None,None,,None,,S,None,,,,,,,,,,,182671554,1018355,687358,None,F496,0,2000-11-07,10001,11001,1999,GPEREZ,11003,11003,None,None,2000-11-07,None
F497,F496,687347,0,2000-11-07,35069467,0,,,,,,,,,,,,,,,,,,SARATOGA,,,099065,REBECCA,COHN,,,4088680249,CA,950700000,,,None,24,None,,,RCP,,OAKLAND,,747285,,"HEALTH CARE WORKERS UNION, SEIU LOCAL 250 POLITICAL ACTION COMMITTEE",,,4085577613,CA,946120000,687347,F497,None,ASM,,,,,,,S,,ASM,,,,CVR,0,,,2000-11-07,None,None,,None,,S,None,,,,,,,,,,,182671546,1020788,687347,None,F496,0,2000-11-07,10001,11001,1999,GGILMORE,11003,11001,None,None,2000-11-07,None
F497,F496,687346,0,2000-11-07,35069466,0,,,,,,,,,,,,,,,,,,SALINAS,,,099033,SIMON,SALINAS,,,8317586276,CA,939050000,,,None,28,None,,,RCP,,OAKLAND,,747285,,"HEALTH CARE WORKERS UNION, SEIU LOCAL 250 POLITICAL ACTION COMMITTEE",,,4085577613,CA,946120000,687346,F497,None,,,,,,,,S,,ASM,,,,CVR,0,,,2000-11-07,None,None,,None,,S,None,,,,,,,,,,,182671545,1020788,687346,None,F496,0,2000-11-07,10001,11001,1999,GGILMORE,11003,11001,None,None,2000-11-07,None
F497,F496,687345,0,2000-11-07,35069465,0,,,,,,,,,,,,,,,,,,ALAMEDA,,,099069,WILMA,CHAN,,,5102726693,CA,945010000,,,None,16,None,,,RCP,,OAKLAND,,747285,,"HEALTH CARE WORKERS UNION, SEIU LOCAL 250 POLITICAL ACTION COMMITTEE",,,4085577613,CA,946120000,687345,F497,None,ASM,,,,,,,S,,ASM,,,,CVR,0,,,2000-11-07,None,None,,None,,S,None,,,,,,,,,,,182766846,1020788,687345,None,F496,0,2000-11-07,10001,11001,1999,GGILMORE,11003,11001,None,None,2000-11-07,None
F497,F496,687216,0,2000-11-07,35070100,0,,,,,,,,SCHOOL VOUCHERS. STATE-FUNDED PRIVATE AND RELIGIOUS EDUCATION. PUBLIC SCHOOL FUNDING. INITIATIVE CONSTITUTIONAL AMENDMENT.,38

### Do any records have conflicting `CVR_CAMPAIGN_DISCLOSURE_CD.FILER_ID` and `FILER_FILINGS_CD.FILER_ID` values?

Yes, this happens about a third of the time.

In [10]:
%%sql
SELECT COUNT(*)::float / (
        SELECT COUNT(*)
        FROM "CVR_CAMPAIGN_DISCLOSURE_CD" CVR
        JOIN "FILER_FILINGS_CD" FF
        ON CVR."FILING_ID" = FF."FILING_ID"
        AND CVR."AMEND_ID" = FF."FILING_SEQUENCE"
) as pct_conflict
FROM "CVR_CAMPAIGN_DISCLOSURE_CD" CVR
JOIN "FILER_FILINGS_CD" FF
ON CVR."FILING_ID" = FF."FILING_ID"
AND CVR."AMEND_ID" = FF."FILING_SEQUENCE"
WHERE CVR."FILER_ID" <> FF."FILER_ID"::VARCHAR;

1 rows affected.


pct_conflict
0.357975298665


But one thing to note is that these `FILER_ID` fields are too different data types: char on `CVR_CAMPAIGN_DISCLOSURE_CD` and int on `FILER_FILINGS_CD`. We previously discovered that the `FILER_XREF_CD` table is a translator from seemingly old string filer_ids to numeric filer_ids. 

## Does every `CVR_CAMPAIGN_DISCLOSURE_CD` record have a `FILER_XREF_ID` record?

No. Here are the missing filer_ids, which are also not found in either the `FILERNAME_CD` or `FILERS_CD` tables.

In [13]:
%%sql
SELECT 
    cvr."FILER_ID" as cvr_filer_id, 
    fn."FILER_ID" as filername_filer_id, 
    f."FILER_ID" as filer_filer_id
FROM (
    SELECT DISTINCT cvr."FILER_ID"
    FROM "CVR_CAMPAIGN_DISCLOSURE_CD" cvr
    LEFT JOIN "FILER_XREF_CD" x
    ON cvr."FILER_ID" = x."XREF_ID"
    WHERE x."XREF_ID" IS NULL
) cvr
LEFT JOIN "FILERNAME_CD" fn
ON cvr."FILER_ID" = fn."FILER_ID"::varchar
LEFT JOIN "FILERS_CD" f
ON cvr."FILER_ID" = f."FILER_ID"::varchar
ORDER BY cvr."FILER_ID"::VARCHAR DESC;

45 rows affected.


cvr_filer_id,filername_filer_id,filer_filer_id
990168,None,None
600230,None,None
499258,None,None
496280,None,None
489049,None,None
486187,None,None
482185,None,None
478533,None,None
1373290,None,None
1372926,None,None


Should probably look more into these later, but this might have something to do with conflicting filer_ids on `CVR_CAMPAIGN_DISCLOSURE_CD` and `FILER_FILINGS_CD`.

### Does the filer_id from `FILER_XREF_CD` and `FILER_FILINGS_CD` ever conflict for the same filing?

Yes, but less than 1 percent of the time.

In [11]:
%%sql
SELECT COUNT(*)::float / (
        SELECT COUNT(*)
        FROM "CVR_CAMPAIGN_DISCLOSURE_CD" CVR
        JOIN "FILER_FILINGS_CD" FF
        ON CVR."FILING_ID" = FF."FILING_ID"
        AND CVR."AMEND_ID" = FF."FILING_SEQUENCE"
) as pct_conflict
FROM "CVR_CAMPAIGN_DISCLOSURE_CD" CVR
JOIN "FILER_FILINGS_CD" FF
ON CVR."FILING_ID" = FF."FILING_ID"
AND CVR."AMEND_ID" = FF."FILING_SEQUENCE"
JOIN "FILER_XREF_CD" X
ON CVR."FILER_ID" = X."XREF_ID"
WHERE X."FILER_ID" <> FF."FILER_ID";

1 rows affected.


pct_conflict
0.00345053087135


Mostly this seems to be a problem for Form 497 filings.

In [12]:
%%sql
SELECT cvr."FORM_TYPE", COUNT(*)
FROM "CVR_CAMPAIGN_DISCLOSURE_CD" CVR
JOIN "FILER_FILINGS_CD" FF
ON CVR."FILING_ID" = FF."FILING_ID"
AND CVR."AMEND_ID" = FF."FILING_SEQUENCE"
JOIN "FILER_XREF_CD" X
ON CVR."FILER_ID" = X."XREF_ID"
WHERE X."FILER_ID" <> FF."FILER_ID"
GROUP BY 1;

2 rows affected.


FORM_TYPE,count
F461,7
F497,1291


In [8]:
%%sql
SELECT COUNT(*)
FROM (
    SELECT "FILING_ID", "AMEND_ID"
    FROM "SMRY_CD"
    GROUP BY 1, 2
) SMRY
LEFT JOIN "FILER_FILINGS_CD" CVR
ON SMRY."FILING_ID" = CVR."FILING_ID"
AND SMRY."AMEND_ID" = CVR."FILING_SEQUENCE"
WHERE CVR."FILING_ID" IS NULL
OR CVR."FILING_SEQUENCE" IS NULL; 

1 rows affected.


count
55
